## Coding Challenge 7: Maps

In this notebook, I use Harvard Atlas ECI to map the Economic Complexity of countries


Link to data:
https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/XTAQMC

In [1]:
%pip install altair   # Some packages need to be installed to the virtual machine before we can import them into our notebook. We can do this with '!pip install'


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pyeucountrycodes


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from eu_country_codes import COUNTRY_CODES
import pandas as pd
import numpy as np
import altair as alt
from json import loads, dumps

### Importing data

In [4]:
eci = pd.read_stata("../data/rankings.dta") # ECI Rankings
country_code = pd.read_csv('../data/location_country.csv') # Country codes

In [5]:
eci.head()

,country_id,year,sitc_eci,hs_eci,growth_proj,sitc_eci_rank,hs_eci_rank
0,4,1980,NaN,0.0,NaN,,72
1,8,1980,NaN,0.0,NaN,,72
2,12,1980,NaN,0.0,NaN,,72
3,24,1980,NaN,0.0,NaN,,72
4,31,1980,NaN,0.0,NaN,,72


In [6]:
country_code.head()

,country_id,name_short_en,iso3_code,legacy_location_id
0,533,Aruba,ABW,0
1,4,Afghanistan,AFG,1
2,24,Angola,AGO,2
3,660,Anguilla,AIA,3
4,8,Albania,ALB,4


### Cleaning 

In [7]:
# Only data from 2000's
#eci = eci[eci['year']>=2000]

# Merge with country code
eci_country_code = pd.merge(eci, country_code, on="country_id")

# Keeping key columns
#eci_country_code = eci_country_code[['year','name_short_en', 'iso3_code','hs_eci_rank']]

eci_country_code

# Renaming columns
eci_country_code.rename(columns={

    "name_short_en" : "country",
    "iso3_code": "ISO3"

}, inplace=True)

eci_country_code

,country_id,year,sitc_eci,hs_eci,growth_proj,sitc_eci_rank,hs_eci_rank,country,ISO3,legacy_location_id
0,4,1980,NaN,0.000000,NaN,,72,Afghanistan,AFG,1
1,8,1980,NaN,0.000000,NaN,,72,Albania,ALB,4
2,12,1980,NaN,0.000000,NaN,,72,Algeria,DZA,66
3,24,1980,NaN,0.000000,NaN,,72,Angola,AGO,2
4,31,1980,NaN,0.000000,NaN,,72,Azerbaijan,AZE,16
...,...,...,...,...,...,...,...,...,...,...
6215,858,2022,0.097432,0.081263,1.494,57,59,Uruguay,URY,230
6216,860,2022,-0.046318,-0.146699,6.148,66,71,Uzbekistan,UZB,232
6217,862,2022,-1.014992,-0.862514,0.937,124,109,Venezuela,VEN,236
6218,887,2022,-0.706749,-1.093413,3.749,103,125,Yemen,YEM,243


### How good was the formation of the EU for the Complexity of European economies?
#### Change in the Complexity index of European countries, from the creation of the EU to now (1993 to 2022). 

In [8]:
iso3 = pd.read_csv('https://raw.githubusercontent.com/lukes/ISO-3166-Countries-with-Regional-Codes/refs/heads/master/all/all.csv')

In [9]:
iso3_eu = iso3[iso3['region']=='Europe']['alpha-3'] # List wih ISO3 of European countries

In [10]:
eci_country_code.head()

,country_id,year,sitc_eci,hs_eci,growth_proj,sitc_eci_rank,hs_eci_rank,country,ISO3,legacy_location_id
0,4,1980,NaN,0.0,NaN,,72,Afghanistan,AFG,1
1,8,1980,NaN,0.0,NaN,,72,Albania,ALB,4
2,12,1980,NaN,0.0,NaN,,72,Algeria,DZA,66
3,24,1980,NaN,0.0,NaN,,72,Angola,AGO,2
4,31,1980,NaN,0.0,NaN,,72,Azerbaijan,AZE,16


In [11]:
# Extract hs_eci for the years 1995 and 2022
country = eci_country_code[eci_country_code['year'] == 1995].set_index('country_id')['country']
ISO3 = eci_country_code[eci_country_code['year'] == 1995].set_index('country_id')['ISO3']
hs_eci_rank_1995 = eci_country_code[eci_country_code['year'] == 1995].set_index('country_id')['hs_eci_rank']
hs_eci_rank_2022 = eci_country_code[eci_country_code['year'] == 2022].set_index('country_id')['hs_eci_rank']

# Combine the data into a single DataFrame
eci_country_code_combined = pd.DataFrame({
    'country': country,
    'ISO3': ISO3,
    'hs_eci_rank_1995': hs_eci_rank_1995,
    'hs_eci_rank_2022': hs_eci_rank_2022
})

# Calculate the percentage change between 1995 and 2022
eci_country_code_combined['hs_eci_rank_1995_2022'] = (eci_country_code_combined['hs_eci_rank_2022'] - eci_country_code_combined['hs_eci_rank_1995'])

# Keeping only EU countries
eci_country_code_combined_eu = eci_country_code_combined[eci_country_code_combined['ISO3'].isin(iso3_eu)]

eci_country_code_combined

,country,ISO3,hs_eci_rank_1995,hs_eci_rank_2022,hs_eci_rank_1995_2022
country_id,,,,,
4,Afghanistan,AFG,91,133,42
8,Albania,ALB,97,85,-12
12,Algeria,DZA,101,103,2
24,Angola,AGO,142,95,-47
31,Azerbaijan,AZE,74,119,45
...,...,...,...,...,...
858,Uruguay,URY,44,59,15
860,Uzbekistan,UZB,88,71,-17
862,Venezuela,VEN,51,109,58


### Growth projections based on Economic Complexity
#### Annual growth (%)

In [12]:
growth_projections = eci_country_code[eci_country_code['year'] == 2022]

growth_projections = growth_projections[['ISO3','country','growth_proj']]

growth_projections.head()

,ISO3,country,growth_proj
6075,AFG,Afghanistan,5.190
6076,ALB,Albania,2.722
6077,DZA,Algeria,2.415
6078,AGO,Angola,5.595
6079,AZE,Azerbaijan,2.983


### Exporting as CSV

In [13]:
# CSV figure 6
growth_projections.to_csv('../data/growth_projections.csv', index=False)  


### Plotting

##### 1) Map

In [16]:
# Load the data
data_url = "https://raw.githubusercontent.com/IgnacioOrueta/ignacioorueta.github.io/refs/heads/main/Project/Section6/data/growth_projections.csv"
data = pd.read_csv(data_url)

data = growth_projections
# Load the geojson data
geojson_url = "https://raw.githubusercontent.com/EconomicsObservatory/courses/main/datasets/advanced/map_world.geojson"
geojson = alt.Data(url=geojson_url, format=alt.DataFormat(property='features', type='json'))

# Create the chart
chart = alt.Chart(data).mark_geoshape(
    stroke='white',
    strokeWidth=0.3
).encode(
    color=alt.condition(
        "!datum['growth_proj']",
        alt.value('lightgrey'),
        alt.Color('growth_proj:Q',
                  legend=alt.Legend(
                      title='Annual growth',
                      format='d',
                      direction='vertical',
                      orient='right',
                      titleFontSize=16,
                      labelFontSize=11,
                      gradientLength=200,
                      labelExpr="datum.value + '%'"
                  ),
                  scale=alt.Scale(
                      type='linear',
                      nice=True,
                      scheme='redblue',
                      reverse=True
                  )
        )
    ),
    tooltip=[
        alt.Tooltip('country:N', title='Country'),
        alt.Tooltip('growth_proj_formatted:N', title='Expected annual growth')
    ]
).transform_lookup(
    lookup='ISO3',
    from_=alt.LookupData(geojson, 'properties.ISO3', ['type', 'geometry', 'properties'])
).transform_calculate(
    growth_proj_formatted="format(datum.growth_proj, '.1f') + '%'"
).project(
    type='naturalEarth1',
    scale=125,
    translate=[285, 180]
).properties(
    width=800,
    height=400,
    title={
        "text": "Economic Complexity Growth Projections to 2032",
        "subtitle": [
            "Projected annualized growth for 2022-2032 (%).",
            "Source: The Harvard Atlas of Economic Complexity.",
            ""
        ],
        "fontSize": 16,
        "subtitleFontSize": 12,
        "subtitleFontWeight": "lighter",
        "anchor": "middle"
    },
    description="Growth projections based on Economic Complexity."
)

chart.save('../figures/figure6_map.json')

chart


alt.Chart(...)